In [ ]:
import tensorflow as tf
import mrcnn.model as modellib
from mrcnn.config import Config
from mrcnn.model import log
from mrcnn import visualize

from tensorflow.python.keras.backend import set_session
sess =  tf.Session()
graph = tf.get_default_graph()

from flask import Flask, request, make_response
import base64
import cv2

DEVICE = "gpu:0" 
weights_path = \
'/dataDisk/myfloder/h5/maskrcnn_snapshots/food20200605T1539/mask_rcnn_food_9500.h5'
MODEL_DIR = '/dataDisk/myfloder/h5/maskrcnn_snapshots/'

class_names = ['BG', '炸排骨', '炒青菜', '辣椒', '杏鮑菇', '蛋', '紅蘿蔔', '炸雞排', '豆腐', '木耳', '花椰菜', '筍絲', '白飯', '青椒', '小黃瓜', '炸雞腿', '玉米', '海帶', '醃蘿蔔', '黑輪', '滷排骨', '香腸', '滷蛋', '醬瓜', '豆干', '瓜仔肉', '酸菜', '雞肉', '雞腿', '浦燒魚', '地瓜', '檸檬', 
     '蟹肉條', '黑豆', '櫻桃', '玉米筍', '青豆仁', '魚板', '炸豬排', '煎魚', '香菇', '甜豆', '炸蝦', '蟹肉棒', '蝦', '燒肉', '芹菜', '彩椒', '烤雞腿', '茄子', '筍干', '爌肉', '櫻花蝦', '豬腳', '蔥', '三色蛋', '甜椒', '魚', '肉鬆', '番茄', '滷牛腱', '火腿', '炸魚排', '蒸蛋', '豆芽菜', 
     '洋芋', '豆皮', '滷雞腿', '洋蔥', '培根', '吻仔魚', '虱目魚', '蝦仁', '鴻喜菇', '炸魚', '菜脯', '苦瓜', '炸雞塊', '冬粉', '肉丸子', '煎鮭魚', '山藥捲', '絲瓜', '酸梅', '筍乾', '麵筋', '四季豆', '炸甜不辣', '花枝排', '麵腸', '九層塔', '大黃瓜', '烤鴨', '白蘿蔔', '麵輪', '叉燒', '油雞', 
     '甜不辣', '烤雞', '煎雞腿', '碗豆', '荸薺', '豬肉', '韭菜', '炸捲', '豆棗', '南瓜', '鹹瓜', '炸雞', '烤鴨腿', '菜脯蛋', '滷肉', '炸蝦捲', '筊白筍', '花枝丸', '花枝', '金針菇', '鳳梨', '蘑菇', '紅燒肉', '章魚', '煎魚排', '丸子', '地瓜球', '鹹鴨蛋', '叉燒肉', '羊排', '小卷', '牛排', 
     '柴魚', '薑', '鯖魚', '雪菜', '毛豆', '煎雞肉', '紫甘藍']

In [ ]:
class InferenceConfig(Config):
    # Give the configuration a recognizable name
    NAME = "food"
    
    # Run detection on one image at a time
    GPU_COUNT = 1
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 135  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    

In [ ]:
# def load_model():
# global model
# global sess

cfg = InferenceConfig()
# Create model in inference mode

with tf.device(DEVICE):    
    model = modellib.MaskRCNN(mode="inference", 
                              model_dir=MODEL_DIR,
                              config=cfg)

model.load_weights(weights_path, by_name=True)
model.keras_model._make_predict_function()
# graph = tf.get_default_graph()

In [ ]:
def print_rois(r,image):
    n=1   
    rois=r['rois']
    plt.figure(1,figsize=(20,20)) 
      
    for reg in rois:
        # print(type(image))
        cropimg = image[reg[0]:reg[2], reg[1]:reg[3], :]
    #     print(type(cropimg))
#         plt.figure(1,figsize=(15,5))
        plt.subplot(math.ceil(len(rois)/5), 5, n)
        
        plt.imshow(cropimg)
    #     plt.show()
        n+=1
              
def calarea(r):
    masks = r['masks']
    ids = r['class_ids']
    print('mask number: {}'.format(np.shape(masks)[2]))
    areas={}

    for iid in set(ids):
        area=0
        for i in range(np.shape(masks)[2]): 
            if ids[i] == iid:
                area = area + np.sum(masks[:,:,i])

        #     mask = np.reshape(mask > .5, (-1, 1)).astype(np.float32)
        #     area = np.sum(mask, axis= 0) #計算mask_面積
        print('mask_area= %d  class_id=%s'%(area,class_names[iid]))
        areas[iid] = area

    r['areas']= areas       

def imgdetect(img_base64): 
    global sess
    global graph
    
    img_data = base64.b64decode(img_base64)
#     img_array = np.fromstring(img_data, np.uint8)
    img_array = np.frombuffer(img_data, np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    
    if image.shape[-1] == 4:    
        image = image[...,:3]
    
    # Run detection
    with graph.as_default():
        set_session(sess)
        results = model.detect([image], verbose=1)

    # Visualize results
    r = results[0]
    visualize.display_instances(image, 
                                r['rois'], 
                                r['masks'], 
                                r['class_ids'], 
                                class_names, 
                                r['scores'])
    

In [ ]:
# from flask import Flask, request, make_response
# import base64
# import cv2

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def get_request():
    if request.method == 'POST':
        content = request.get_json()
#         img_data = base64.b64decode(content['photo']) 
#         img_array = np.fromstring(img_data, np.uint8)
#         img = cv2.imdecode(img_array, cv2.IMREAD_COLOR) 
        img_base64 = content['photo']
        imgdetect(img_base64)
        
#         response = make_response(img_data)
#         response.headers['Content-Type'] = 'image/jpg'

#         return response

if __name__ == '__main__':
    app.run(host='140.116.67.155', port=5000)

In [ ]:
content = request.get_json()

In [ ]:
from flask import Flask, request, make_response
from datetime import datetime
import os

import time
import base64
import cv2
 
app = Flask(__name__)

@app.route('/img/<string:filename>', methods=['POST', 'GET'])
def display_img(filename):
    request_begin_time = datetime.today()
    print("request_begin_time", request_begin_time)
    
    if request.method == 'POST':       
        content = request.get_json()
        
        img_data = base64.b64decode(content['photo'])
        img_array = np.fromstring(img_data, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR) 
        
        filename = time.strftime('%Y%m%d%H%M')
        print(filename)
      
        response = make_response(img)
#         response.headers['Content-Type'] = 'image/jpg'
        return url_for('display_img',filename=filename)
   
if __name__ == '__main__':
    app.run(host='140.116.67.155', port=5000)

In [ ]:
from flask import Flask, request, make_response
from datetime import datetime
import os
 
app = Flask(__name__)
IMG_PATH = "./img/"
 
 
@app.route('/display/img/<string:filename>', methods=['GET'])
def display_img(filename):
    request_begin_time = datetime.today()
    print("request_begin_time", request_begin_time)
    if request.method == 'GET':
        if filename is None:
            pass
        else:
            image_data = open(IMG_PATH + filename, "rb").read()
            print(type(image_data))
            response = make_response(image_data)
            response.headers['Content-Type'] = 'image/jpg'
            return response
    else:
        pass
 
 
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, make_response,jsonify
from werkzeug.utils import secure_filename
import os
import cv2
import time
 
from datetime import timedelta
 
#设置允许的文件格式
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'JPG', 'PNG', 'bmp'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS


In [ ]:
app = Flask(__name__)
# 设置静态文件缓存过期时间
app.send_file_max_age_default = timedelta(seconds=1)

In [ ]:
# @app.route('/upload', methods=['POST', 'GET'])
@app.route('/upload', methods=['POST', 'GET'])  # 添加路由
def upload():
    if request.method == 'POST':
        f = request.files['file']
        if not (f and allowed_file(f.filename)):
            return jsonify({"error": 1001, "msg": "请检查上传的图片类型，仅限于png、PNG、jpg、JPG、bmp"})
        
        user_input = request.form.get("name")
        
        basepath = os.path.dirname(__file__)  # 当前文件所在路径
        
        upload_path = os.path.join(basepath, 'static/images', secure_filename(f.filename))  # 注意：没有的文件夹一定要先创建，不然会提示没有该路径
        # upload_path = os.path.join(basepath, 'static/images','test.jpg')  #注意：没有的文件夹一定要先创建，不然会提示没有该路径
        f.save(upload_path)
        
        # 使用Opencv转换一下图片格式和名称
        img = cv2.imread(upload_path)
        cv2.imwrite(os.path.join(basepath, 'static/images', 'test.jpg'), img)
        
        return render_template('upload_ok.html',userinput=user_input,val1=time.time())
    
    return render_template('upload.html')


In [ ]:
if __name__ == '__main__':
    # app.debug = True
    app.run(host='0.0.0.0', port=8987, debug=True)

In [ ]:
img_data = base64.b64decode(base64_code)
img_array = np.fromstring(img_data, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR) 

In [ ]:
from flask import Flask
app = Flask(__name__)

In [ ]:
@app.route("/")
def hello():
    return "Hello Flask!"

@app.route("/apple")
def apple():
    return "I am apple!"

if __name__ == "__main__":
    app.run()

In [ ]:
# @app.route('/upload', methods=['POST', 'GET'])
@app.route('/upload', methods=['POST', 'GET'])  # 添加路由
def upload():
    if request.method == 'POST':
        f = request.files['file']
        if not (f and allowed_file(f.filename)):
            return jsonify({"error": 1001, "msg": "请检查上传的图片类型，仅限于png、PNG、jpg、JPG、bmp"})
        
        user_input = request.form.get("name")
        
        basepath = os.path.dirname(__file__)  # 当前文件所在路径
        
        upload_path = os.path.join(basepath, 'static/images', secure_filename(f.filename))  # 注意：没有的文件夹一定要先创建，不然会提示没有该路径
        # upload_path = os.path.join(basepath, 'static/images','test.jpg')  #注意：没有的文件夹一定要先创建，不然会提示没有该路径
        f.save(upload_path)
        
        # 使用Opencv转换一下图片格式和名称
        img = cv2.imread(upload_path)
        cv2.imwrite(os.path.join(basepath, 'static/images', 'test.jpg'), img)
        
        return render_template('upload_ok.html',userinput=user_input,val1=time.time())
    
    return render_template('upload.html')
